In [1]:
from pyspark.sql import SparkSession, types as T, functions as F
from pyspark import SparkConf

conf = SparkConf().setAppName("learn")
conf.set('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f53ef31b-3c0a-4b83-a4b5-b14309d80d40;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 595ms :: artifacts dl 25ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnl

In [3]:
schema_redemer = T.StructType([T.StructField(nm, T.StringType(), True) for nm in ['Sr_No', 'Date_of_Encashment', 'Name_of_the_Political_Party', 'Account_no._of_Political_Party', 'Prefix', 'Bond_Number', 'redemer_Denominations', 'Pay_Branch_Code', 'Pay_Teller'] ])
schema_purchaser = T.StructType([T.StructField(nm, T.StringType(), True) for nm in ['Sr_No', 'Reference_No_(URN)', 'Journal_Date', 'Date_of_Purchase', 'Date_of_Expiry', 'Name_of_the_Purchaser', 'Prefix', 'Bond_Number', 'purchaser_Denominations', 'Issue_Branch_Code', 'Issue_Teller', 'Status'] ])
df_redemer = spark.read.format('csv').schema(schema_redemer).load('/home/glue_user/workspace/data-engineering/data/source/src-data-csv/electoral-bonds/bond_redemer_details.csv').filter("Pay_Teller is not null")
df_purchaser = spark.read.format('csv').schema(schema_purchaser).load('/home/glue_user/workspace/data-engineering/data/source/src-data-csv/electoral-bonds/bond_purchaser_details.csv').filter("purchaser_Denominations is not null")

df_redemer.printSchema()
df_purchaser.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date_of_Encashment: string (nullable = true)
 |-- Name_of_the_Political_Party: string (nullable = true)
 |-- Account_no._of_Political_Party: string (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- Bond_Number: string (nullable = true)
 |-- redemer_Denominations: string (nullable = true)
 |-- Pay_Branch_Code: string (nullable = true)
 |-- Pay_Teller: string (nullable = true)

root
 |-- Sr_No: string (nullable = true)
 |-- Reference_No_(URN): string (nullable = true)
 |-- Journal_Date: string (nullable = true)
 |-- Date_of_Purchase: string (nullable = true)
 |-- Date_of_Expiry: string (nullable = true)
 |-- Name_of_the_Purchaser: string (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- Bond_Number: string (nullable = true)
 |-- purchaser_Denominations: string (nullable = true)
 |-- Issue_Branch_Code: string (nullable = true)
 |-- Issue_Teller: string (nullable = true)
 |-- Status: string (nullable = true)



In [23]:
df_redemer.select(F.expr("sum(cast(replace(redemer_Denominations, ',', '') as bigint)) over(partition by Name_of_the_Political_Party)").alias('party_wise_donation'),
                    F.expr("Name_of_the_Political_Party")).show()

+-------------------+---------------------------+
|party_wise_donation|Name_of_the_Political_Party|
+-------------------+---------------------------+
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|
|          654500000|            AAM AADMI PARTY|


In [92]:
def str_to_inr(s: str) -> str :
    n = int(s)
    q = n%1000
    s1 = str(n%1000)
    n = int(n/1000)
    s2 = ''
    while(n):
        r = n%100
        n = int(n/100)
        s2 = str(r) + ',' + s2
    return s2+s1
str_to_inr = F.udf(str_to_inr, T.StringType())
spark.udf.register("str_to_inr", str_to_inr)

24/05/23 13:45:00 WARN SimpleFunctionRegistry: The function str_to_inr replaced a previously registered function.


<function __main__.str_to_inr(s: str) -> str>

In [109]:
df_redemer.withColumn('redemer_Denominations', 
                      F.expr("cast(replace(redemer_Denominations, ',', '') as bigint)")).orderBy(F.expr('redemer_Denominations').desc()).show()

+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+
|Sr_No|Date_of_Encashment|Name_of_the_Political_Party|Account_no._of_Political_Party|Prefix|Bond_Number|redemer_Denominations|Pay_Branch_Code|Pay_Teller|
+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+
|   34|       12/Apr/2019|       BHARAT RASHTRA SA...|                   *******7477|    OC|       6646|             10000000|          00847|   3367975|
|   54|       12/Apr/2019|       BHARAT RASHTRA SA...|                   *******7477|    OC|       5457|             10000000|          00847|   3367975|
|   35|       12/Apr/2019|       BHARAT RASHTRA SA...|                   *******7477|    OC|       6644|             10000000|          00847|   3367975|
|    2|       12/Apr/2019|       ALL INDIA ANNA DR...|                   ***

In [5]:
# Define the schema for each table
users_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("name", T.StringType(), True)
])

addresses_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("street", T.StringType(), True),
    T.StructField("city", T.StringType(), True),
    T.StructField("postal_code", T.StringType(), True)
])

favorite_products_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("product_id", T.StringType(), True)
])

preferences_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("key", T.StringType(), True),
    T.StructField("value", T.StringType(), True)
])

extra_info_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("field1", T.StringType(), True),
    T.StructField("field2", T.StringType(), True)
])

additional_info_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("field3", T.StringType(), True),
    T.StructField("field4", T.StringType(), True)
])

# Define data for each table
users_data = [
    ("U123", "John Doe"),
    ("U124", "Jane Doe"),
    ("U125", "Alice Smith"),
    ("U126", "Bob Brown")
]

addresses_data = [
    ("U123", "123 Elm St", "Somewhere", "12345"),
    ("U123", "456 Oak St", "Anywhere", "67890"),
    ("U124", "789 Pine St", "Somewhere", "12345"),
    ("U125", "101 Maple St", "Elsewhere", "54321"),
    ("U126", "202 Birch St", "Nowhere", "98765")
]

favorite_products_data = [
    ("U123", "P001"),
    ("U123", "P002"),
    ("U123", "P003"),
    ("U124", "P004"),
    ("U124", "P005"),
    ("U125", "P006"),
    ("U126", "P007"),
    ("U126", "P008"),
    ("U126", "P009"),
    ("U126", "P010")
]

preferences_data = [
    ("U123", "newsletter", "subscribed"),
    ("U123", "theme", "dark"),
    ("U124", "newsletter", "unsubscribed"),
    ("U124", "theme", "light"),
    ("U124", "language", "en"),
    ("U125", "theme", "dark"),
    ("U126", "language", "fr"),
    ("U126", "currency", "euro"),
    ("U126", "newsletter", "subscribed")
]

extra_info_data = [
    ("U123", "foo", "bar"),
    ("U124", "baz", "qux"),
    ("U125", "abc", "def"),
    ("U126", "ghi", "jkl")
]

additional_info_data = [
    ("U123", "foo1", "bar1"),
    ("U124", "baz1", "qux1"),
    ("U125", "abc1", "def1"),
    ("U126", "ghi1", "jkl1")
]

# Create DataFrames for each table
users_df = spark.createDataFrame(users_data, schema=users_schema)
addresses_df = spark.createDataFrame(addresses_data, schema=addresses_schema)
favorite_products_df = spark.createDataFrame(favorite_products_data, schema=favorite_products_schema)
preferences_df = spark.createDataFrame(preferences_data, schema=preferences_schema)
extra_info_df = spark.createDataFrame(extra_info_data, schema=extra_info_schema)
additional_info_df = spark.createDataFrame(additional_info_data, schema=additional_info_schema)

# Show DataFrames
print("users:")
users_df.show()

print("addresses:")
addresses_df.show()

print("favorite_products:")
favorite_products_df.show()

print("favorite_products:")
preferences_df.show()

print("extra_info:")
extra_info_df.show()

print("additional_info:")
additional_info_df.show()

users_df.createOrReplaceTempView("users")
addresses_df.createOrReplaceTempView("addresses")
favorite_products_df.createOrReplaceTempView("favorite_products")
preferences_df.createOrReplaceTempView("preferences")
extra_info_df.createOrReplaceTempView("extra_info")
additional_info_df.createOrReplaceTempView("additional_info")

users:
+-------+-----------+
|user_id|       name|
+-------+-----------+
|   U123|   John Doe|
|   U124|   Jane Doe|
|   U125|Alice Smith|
|   U126|  Bob Brown|
+-------+-----------+

addresses:
+-------+------------+---------+-----------+
|user_id|      street|     city|postal_code|
+-------+------------+---------+-----------+
|   U123|  123 Elm St|Somewhere|      12345|
|   U123|  456 Oak St| Anywhere|      67890|
|   U124| 789 Pine St|Somewhere|      12345|
|   U125|101 Maple St|Elsewhere|      54321|
|   U126|202 Birch St|  Nowhere|      98765|
+-------+------------+---------+-----------+

favorite_products:
+-------+----------+
|user_id|product_id|
+-------+----------+
|   U123|      P001|
|   U123|      P002|
|   U123|      P003|
|   U124|      P004|
|   U124|      P005|
|   U125|      P006|
|   U126|      P007|
|   U126|      P008|
|   U126|      P009|
|   U126|      P010|
+-------+----------+

favorite_products:
+-------+----------+------------+
|user_id|       key|       value

In [98]:
sqlQuery = '''
select
    users.user_id,
    array(
        street,
        city,
        postal_code
    ) address
from
    users
    left join addresses on users.user_id=addresses.user_id
    left join favorite_products on favorite_products.user_id=addresses.user_id
'''
df_output = spark.sql(sqlQuery)
df_output.show(truncate=False)
df_output.printSchema()
df_output.toJSON().collect()
df_output.createOrReplaceTempView('op')

+-------+--------------------------------+
|user_id|address                         |
+-------+--------------------------------+
|U124   |[789 Pine St, Somewhere, 12345] |
|U124   |[789 Pine St, Somewhere, 12345] |
|U125   |[101 Maple St, Elsewhere, 54321]|
|U126   |[202 Birch St, Nowhere, 98765]  |
|U126   |[202 Birch St, Nowhere, 98765]  |
|U126   |[202 Birch St, Nowhere, 98765]  |
|U126   |[202 Birch St, Nowhere, 98765]  |
|U123   |[456 Oak St, Anywhere, 67890]   |
|U123   |[456 Oak St, Anywhere, 67890]   |
|U123   |[456 Oak St, Anywhere, 67890]   |
|U123   |[123 Elm St, Somewhere, 12345]  |
|U123   |[123 Elm St, Somewhere, 12345]  |
|U123   |[123 Elm St, Somewhere, 12345]  |
+-------+--------------------------------+

root
 |-- user_id: string (nullable = true)
 |-- address: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [100]:
spark.sql('''
select *, explode(address) from op          
''').show()

+-------+--------------------+------------+
|user_id|             address|         col|
+-------+--------------------+------------+
|   U124|[789 Pine St, Som...| 789 Pine St|
|   U124|[789 Pine St, Som...|   Somewhere|
|   U124|[789 Pine St, Som...|       12345|
|   U124|[789 Pine St, Som...| 789 Pine St|
|   U124|[789 Pine St, Som...|   Somewhere|
|   U124|[789 Pine St, Som...|       12345|
|   U125|[101 Maple St, El...|101 Maple St|
|   U125|[101 Maple St, El...|   Elsewhere|
|   U125|[101 Maple St, El...|       54321|
|   U126|[202 Birch St, No...|202 Birch St|
|   U126|[202 Birch St, No...|     Nowhere|
|   U126|[202 Birch St, No...|       98765|
|   U126|[202 Birch St, No...|202 Birch St|
|   U126|[202 Birch St, No...|     Nowhere|
|   U126|[202 Birch St, No...|       98765|
|   U126|[202 Birch St, No...|202 Birch St|
|   U126|[202 Birch St, No...|     Nowhere|
|   U126|[202 Birch St, No...|       98765|
|   U126|[202 Birch St, No...|202 Birch St|
|   U126|[202 Birch St, No...|  

24/05/29 07:10:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 51792632 ms exceeds timeout 120000 ms
24/05/29 07:10:36 WARN SparkContext: Killing executors is not supported by current scheduler.


In [48]:
sqlQuery = '''
SELECT
    c2,
    collect_list(c1)
FROM (
    VALUES 
        (1, 'A'),
        (1, 'A'),
        (1, 'A'),
        (2, 'B'),
        (2, 'B'), 
        (3, 'C')
) AS tab(c1, c2)
group by c2
'''
spark.sql(sqlQuery).printSchema()

root
 |-- c2: string (nullable = false)
 |-- collect_list(c1): array (nullable = false)
 |    |-- element: integer (containsNull = false)



In [53]:
sqlQuery = '''
SELECT
    map(c1, c2) cl
FROM (
    VALUES 
        (1, 'A'),
        (1, 'A'),
        (1, 'A'),
        (2, 'B'),
        (2, 'B'), 
        (3, 'C')
) AS tab(c1, c2)
'''
spark.sql(sqlQuery).show()

+--------+
|      cl|
+--------+
|{1 -> A}|
|{1 -> A}|
|{1 -> A}|
|{2 -> B}|
|{2 -> B}|
|{3 -> C}|
+--------+



In [57]:
# Create sample DataFrames
df1 = spark.createDataFrame([("A", 1), ("B", 2), ("C", 3)], ["id", "value1"])
df2 = spark.createDataFrame([("A", 4), ("B", 5), ("D", 6)], ["id", "value2"])

# Join using expr for complex condition
df1.join()

AnalysisException: Column 'df1.value1' does not exist. Did you mean one of the following? [value1, value2, id, id]; line 1 pos 0;
'Join Inner, (('df1.value1 * 2) = 'df2.value2)
:- LogicalRDD [id#698, value1#699L], false
+- LogicalRDD [id#702, value2#703L], false


In [87]:
# Sample data with nested structure
data = [(1, [1, 2, 3]), (2, [4, 5])]

# Create DataFrame
df = spark.createDataFrame(data, ["id", "values"])

# Explode the array column
df.show()
df_flattened = df.select("id", F.explode("values").alias("value"))

# Show the flattened DataFrame
df_flattened.show()

+---+---------+
| id|   values|
+---+---------+
|  1|[1, 2, 3]|
|  2|   [4, 5]|
+---+---------+

+---+-----+
| id|value|
+---+-----+
|  1|    1|
|  1|    2|
|  1|    3|
|  2|    4|
|  2|    5|
+---+-----+



In [3]:
#corrupted records
df = spark.read.format('csv').options(schema=['id', 'name', '_corrupt_records'], header='true', mode='PERMISSIVE', badRecordsPath='/home/glue_user/workspace/data-engineering/tmp/badrecords').load('C:\Users\raulr\OneDrive\myProjects\dockerBind\awsglue\data-engineering\data\source\src-data-csv-corrupted\actor.csv')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3111779085.py, line 2)

In [140]:
df.show()

+-----+---------------+
|   id|           name|
+-----+---------------+
|30509|      Tori Amos|
|35329|Barbara LeakeVB|
|12445|       Don Pike|
+-----+---------------+



In [4]:
class Animal():
    def __init__(self, hands, legs):
        self.hands = hands
        self.legs = legs
    def run(self, speed):
        return f"running at speed {speed}"

In [6]:
human = Animal(2, 2)
human.run(10)

'running at speed 10'